In [244]:
import config
import json
import os
import random
import pickle
random.seed(1234)
import numpy as np; np.random.seed(1234)
from utils_picking import my_read_old, round_percent, vqa_score_list

In [ ]:
TEST_SPLIT = "val2014"

MODEL = "SNMN"   # alternatively can put "SAAA", "CNN_LSTM" 

#note: for testing- we test on those IQAs from val where 10 answers are uniform

In [247]:
orig_root_dir_qa = 'testing_iv_vqa/original'
edit_root_dir_qa = 'testing_iv_vqa/edited'
coco_json = 'coco_cat_ids.json'

#### questions keys: 'image_id', 'question', 'question_id'
res_file_q = 'v2_OpenEnded_mscoco_' + TEST_SPLIT + '_questions.json'
standard_questions_val_json = os.path.join(orig_root_dir_qa, res_file_q)
standard_questions_edit_val_json = os.path.join(edit_root_dir_qa, res_file_q)

## ann keys: 'image_id', 'question_id', 'answers' , 'multiple_choice_answer'(the most frequent answer), 'question_type', 'answer_type'
res_file_a = 'v2_mscoco_' + TEST_SPLIT + '_annotations.json'
standard_annotations_val_json = os.path.join(orig_root_dir_qa, res_file_a)
standard_annotations_edit_val_json = os.path.join(edit_root_dir_qa, res_file_a)

In [248]:
### make changes here
if MODEL == 'SNMN':
    results_edit_val= config.results_edit_val_snmn
    results_val= config.results_val_snmn
    ## standard_vocab file_model_specific
    standard_vocab_ans_file = config.standard_vocab_ans_file_snmn
    with open(standard_vocab_ans_file) as f:
        ans_vocab_list = f.read().splitlines()
    
elif MODEL == 'SAAA':
    results_edit_val = config.results_edit_val_saaa
    results_val = config.results_val_saaa
    ## standard_vocab file_model_specific
    vocab = config.standard_vocab_ans_file_saaa
    with open(vocab, 'r') as f:
        ans_vocab = json.load(f)["answer"]
        ans_vocab_list = [k for k, v in ans_vocab.items()]
        #{v: k for k, v in ans_vocab.items()}   ### is a dictionary here but will work: keys- index- 0,1,2...

elif MODEL == 'CNN_LSTM':
    results_edit_val = config.results_edit_val_cnn_lstm
    results_val = config.results_val_cnn_lstm
    ## standard_vocab file_model_specific
    vocab = config.standard_vocab_ans_file_cnn_lstm
    with open(vocab, 'r') as f:
        ans_vocab = json.load(f)["answer"]
        ans_vocab_list = [k for k, v in ans_vocab.items()]
        # {v: k for k, v in ans_vocab.items()}   ### is a dictionary here but will work: keys- index- 0,1,2...
    

In [252]:
qid_val, pred_ans_val , ss_vc_val, img_ids_val, ques_val, all_ans_val, ques_type_val, ans_type_val = my_read_old(results_val, standard_questions_val_json, standard_annotations_val_json)

qid_edit_val, pred_ans_edit_val , ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val = my_read_old(results_edit_val, standard_questions_edit_val_json, standard_annotations_edit_val_json)

masking_indices_where_qid_edit_but_no_orig_qid = [idx for idx, i in enumerate(qid_edit_val) if i not in qid_val]
stop_idx = masking_indices_where_qid_edit_but_no_orig_qid[0]
qid_edit_val = qid_edit_val[0:stop_idx]
qid_edit_val, pred_ans_edit_val, ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val = [
    i[0:stop_idx] for i in
    [qid_edit_val, pred_ans_edit_val, ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val,
     ques_type_edit_val, ans_type_edit_val]]
print(stop_idx)


6.4199817180633545


In [256]:
len_val = len(img_ids_val)
len_edit_val = len(img_ids_edit_val)
all_indices_val = np.arange(len_val)
all_indices_edit_val = np.arange(len_edit_val)

print('size of original set:', len_val)
print('size of edited set:', len_edit_val)
print('no of unique images in val_set: ',len(set(img_ids_val)))
print('no of unique images in edit_val_set: ',len(set(img_ids_edit_val)))
print('#unique questions in original set: ',len(list(set(qid_val))) )
print('#unique questions in edited set: ', len(list(set(qid_edit_val))))
    
if len(list(set(qid_val))) != len(list(set(qid_edit_val))) :
    print('not every question in orig_val made it to edit_val=> for ', len(list(set(qid_val)))- len(list(set(qid_edit_val)))  ,' questions- no legit edited IQA possible')

qid_gt_ans_label = {}
for idx,a in enumerate(qid_val):
    qid_gt_ans_label[a] = all_ans_val[idx]
    
qid_predans_val = {}
for idx, a in enumerate(qid_val):
    qid_predans_val.setdefault(a, []).append(pred_ans_val[idx])
    
qid_predans_idx_val = {}
for idx, a in enumerate(qid_val):
    qid_predans_idx_val.setdefault(a, []).append(idx)
    
#idx in case here refers to len(val and edit_val- order hai - so relax)    
qid_predans_edit_val = {}
for idx, a in enumerate(qid_edit_val):
    qid_predans_edit_val.setdefault(a, []).append(pred_ans_edit_val[idx])
    #qid_predans_edit_val[a] = (pred_ans_edit_val[idx])   
    
qid_predans_idx_edit_val = {}
for idx, a in enumerate(qid_edit_val):
    qid_predans_idx_edit_val.setdefault(a, []).append(idx)    
    
# qid_predans_imgid_edit_val = {}
# for idx, a in enumerate(qid_edit_val):
#     qid_predans_imgid_edit_val.setdefault(a, []).append(img_ids_edit_val[idx])   

## creating dictionary for val set - to facilitate extensions based on q_id index
qid_ss_predans_val = {}
for idx, a in enumerate(qid_val):
    qid_ss_predans_val[a] = (ss_vc_val[idx], pred_ans_val[idx], qid_val[idx],idx, all_ans_val[idx])

extended_ss_vc_val = [qid_ss_predans_val[q_id][0] for q_id in qid_edit_val]
extended_pred_ans_val = [qid_ss_predans_val[q_id][1] for q_id in qid_edit_val] 
#extended_qid_val = [qid_ss_predans_val[q_id][2] for q_id in qid_edit_val]

collapsed_pred_ans_val = [qid_ss_predans_val[q_id][1] for q_id in set(qid_edit_val)] 
collapsed_all_ans_val = [qid_ss_predans_val[q_id][4] for q_id in set(qid_edit_val)] 
collapsed_indices = [qid_ss_predans_val[q_id][3] for q_id in set(qid_edit_val)] 



size of original set: 63219
size of edited set: 108239
no of unique images in val_set:  29376
no of unique images in edit_val_set:  53855
#unique questions in original set:  63219
#unique questions in edited set:  45076
not every question in orig_val made it to edit_val=> for  18143  questions- no legit edited IQA possible


In [258]:
###                                         STATISTICS
assert len(collapsed_indices) == len(set(qid_edit_val))
ind_where_10_ans_same = [idx for idx, i in enumerate(all_ans_val) if len(set(i))==1 ]  
accuracy_ind_bf_10_ans_same = [i for i in ind_where_10_ans_same if ans_vocab_list[pred_ans_val[i]] in all_ans_val[i]]

##official_way
off_score_val = vqa_score_list(all_ans_val, pred_ans_val, ans_vocab_list)
off_score_val_extended = vqa_score_list(all_ans_edit_val, extended_pred_ans_val, ans_vocab_list)
off_score_val_collapsed = vqa_score_list(collapsed_all_ans_val, collapsed_pred_ans_val, ans_vocab_list)
off_score_edit_val = vqa_score_list(all_ans_edit_val, pred_ans_edit_val, ans_vocab_list)
print('official accuracy before editing', round_percent(np.sum(off_score_val) / len_val))
print('official accuracy before editing_all_10_ans_same', round_percent(len(accuracy_ind_bf_10_ans_same) / len(ind_where_10_ans_same)), len(ind_where_10_ans_same))
print('official accuracy before editing_extended', round_percent(np.sum(off_score_val_extended) / len_edit_val))
print('official accuracy after editing', round_percent(np.sum(off_score_edit_val) / len_edit_val))
print('official accuracy before editing_collapsed', round_percent(np.sum(off_score_val_collapsed) / len(collapsed_indices)))


accuracy before editing, one answer match is good 66.039 41749
accuracy before editing_10_ans_same, one answer match is good 66.039 63219
accuracy before editing_extended 65.447 70839
accuracy after editing, one answer match is good 65.146 70513
accuracy before editing_collapsed 65.671
worst case accuracy (45076):  63.178
best case accuracy (45076):  67.517
worst case official accuracy (45076):  63.178
best case official accuracy (45076):  67.517
official accuracy before editing 66.039
official accuracy before editing_all_10_ans_same 66.039 63219
official accuracy before editing_extended 65.447
official accuracy after editing 65.146
official accuracy before editing_collapsed 65.671


In [259]:
### for entire set counting how mnay labels flipped
labels_flipped_count = np.sum([extended_pred_ans_val[i] != val for i, val in enumerate(pred_ans_edit_val)])
labels_remained_same_count = np.sum([extended_pred_ans_val[i] == val for i, val in enumerate(pred_ans_edit_val)])
assert (labels_flipped_count + labels_remained_same_count == len(pred_ans_edit_val))
print('#labels flipped for entire set=', labels_flipped_count,'      ' ,
      round_percent(labels_flipped_count / len_edit_val), '%')
      
print('#labels remained same=', labels_remained_same_count,'       ', 
      round_percent(labels_remained_same_count / len_edit_val), '%')

lab_fl_ind = [i for i in range(len_edit_val) if extended_pred_ans_val[i] != pred_ans_edit_val[i]]
# label i.e ans was wrong before- right now- one match to 10gt ans is okay
lab_fl_pos = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] not in all_ans_edit_val[i] and ans_vocab_list[pred_ans_edit_val[i]] in all_ans_edit_val[i]]
# label i.e ans was right before- now wrong - one match to 10gt ans is okay
lab_fl_neg = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] in all_ans_edit_val[i] and ans_vocab_list[pred_ans_edit_val[i]] not in all_ans_edit_val[i]]
lab_fl_right = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] in all_ans_edit_val[i] and ans_vocab_list[pred_ans_edit_val[i]] in all_ans_edit_val[i]]
lab_fl_wrong = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] not in all_ans_edit_val[i] and ans_vocab_list[pred_ans_edit_val[i]] not in all_ans_edit_val[i]]
print('#labels flipped that show a positive change', len(lab_fl_pos), '         ', 
      round_percent(len(lab_fl_pos) / len_edit_val), '%' )
print('#labels flipped that show a negative change', len(lab_fl_neg), '         ', 
      round_percent(len(lab_fl_neg) / len_edit_val), '%')
print('#labels flipped that are both right', len(lab_fl_right), '         ', round_percent(len(lab_fl_right) / len_edit_val), '%')
print('#labels flipped that are both wrong', len(lab_fl_wrong), '         ', round_percent(len(lab_fl_wrong) / len_edit_val), '%')
assert(len(lab_fl_pos) + len(lab_fl_neg) + len(lab_fl_right) + len(lab_fl_wrong) == len(lab_fl_ind))


lab_sm_ind = [i for i in range(len_edit_val) if extended_pred_ans_val[i] == pred_ans_edit_val[i]]
# labels that remained exactly same- so two cases possible- either right/wrong
lab_sm_right = [i for i in lab_sm_ind if ans_vocab_list[pred_ans_edit_val[i]] in all_ans_edit_val[i]]
lab_sm_wrong = [i for i in lab_sm_ind if ans_vocab_list[pred_ans_edit_val[i]] not in all_ans_edit_val[i]]
print('#labels same that are both right', len(lab_sm_right), '         ', round_percent(len(lab_sm_right)/len_edit_val), '%')
print('#labels same that are both wrong', len(lab_sm_wrong), '         ', round_percent(len(lab_sm_wrong) / len_edit_val), '%')
assert (len(lab_sm_right) + len(lab_sm_wrong) == len(lab_sm_ind))

#labels flipped for entire set= 7059        6.522 %
#labels remained same= 101180         93.478 %
#labels flipped that show a positive change 2758           2.548 %
#labels flipped that show a negative change 3084           2.849 %
#labels flipped that are both right 0           0.0 %
#labels flipped that are both wrong 1217           1.124 %
#labels same that are both right 67755           62.598 %
#labels same that are both wrong 33425           30.881 %
